# RNN for sentence generation

## 단어 단위(Word-level)의 RNN 구현해보자

### <학습목표>
1. 이번 노트북에서는 단어(word) 단위의 입력값으로 RNN을 학습해 보고, 결과를 Tensorboard를 이용하여 보는 것을 목표로 합니다.
2. 학습할 데이터는 Sherlock homes 시리즈 중 The Sign of the Four의 영문책을 이용하여 학습합니다.
3. 학습된 모델을 이용하여 새로운 문장을 만들어 봅니다.

## Hyperparameters

위에 선언한 함수에서 이제 hyperparameter들을 정합니다. 
일반적으로 network의 크기가 커질 수록(hidden unit이 많을 수록, layer 수가 많을 수록) 성능이 향상되지만, overfitting(fit to variance)이 되는 현상을 잘 관찰해야 합니다. hyperparameter들이 너무 적을 경우에는 underfitting(fit to bias)되는 현상이 있을 수 있습니다.

In [1]:
params = {
    'lstm_size' : 128,
    'batch_size': 100,
    'time_steps': 15,    
    'num_layers' : 3,
    'optimizer_params': {'learning_rate': 1e-3}}

Dependencies 읽기

In [2]:
# python2 -- python3
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from collections import namedtuple
from six.moves import urllib

import time
import re
import os
import numpy as np
import tensorflow as tf
import tensorflow.contrib.rnn as rnn

In [3]:
# Download the data.
url = 'http://cvlab.postech.ac.kr/~wgchang/data/others/'

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        if not os.path.isdir(os.path.dirname(filename)):
            os.makedirs(os.path.dirname(filename))
        filename, _ = urllib.request.urlretrieve(url + os.path.basename(filename), filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception(
            'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename


In [4]:
filename = maybe_download('../data/sherlock.txt', 3377296)
# filename = maybe_download('../data/sherlock_short.txt', 609394)

Found and verified ../data/sherlock.txt


Tensorflow GPU settings

In [5]:
# configuration for prevent whole gpu usage
config = tf.ConfigProto()
config.gpu_options.allow_growth=True

우선 텍스트 데이터를 불러들인 후 각 단어들을 정수값으로 변환하여 모델이 학습할 수 있도록 합니다.

In [6]:
def remove_multiple_s(text):
    # 여러번 띄어쓰기가 된 부분을 한번으로 수정합니다.
    text = re.sub(r' +',r' ',text)
    # 여러번 탭이 된 부분을 한번으로 수정합니다.
    text = re.sub(r'\t+',r' ',text)
    # 여러번 newline으로된 부분을 한번으로 수정합니다.
    text = re.sub(r'\n+',r' ',text)
    # 특수문자를 제거합니다.
    text = re.sub(r'[^A-Za-z0-9.,\?!\'" ]+',r'',text)
    return text

In [7]:
with open(filename, 'r') as f:
    text=f.read()
text=remove_multiple_s(text.lower())
texts = re.findall(r'([.,\?!\'"0-9]|\b[a-zA-z]+\b)',text,re.IGNORECASE)
dictionary = set(texts)
word_to_int = {c: i for i, c in enumerate(dictionary)}
int_to_word = dict(enumerate(dictionary))
words = np.array([word_to_int[c] for c in texts], dtype=np.int32)

In [8]:
words

array([16387,  9801, 15499, ...,  6579,   848, 13063], dtype=int32)

텍스트의 길이와 텍스트가 숫자로 변환되었는지 확인합니다.

In [9]:
len(words)

693263

In [10]:
len(dictionary)

20372

In [11]:
texts[:10]

['chapter', 'i', 'mr', '.', 'sherlock', 'holmes', 'in', 'the', 'year', '1']

이제 데이터를 training과 validation으로 나누고 각각을 batch로 만들어봅시다. 이번 과제에서는 Test set은 따로 없습니다.
문장에서 input과 target의 배열을 만듭니다. 여기서 target은 input과 같은 길이의 글자열이지만 한 글자가 밀려진 글자열입니다.
batch 크기를 맞추기 위해서 문장의 뒤에 남는 부분은 버립니다.
split_frac은 training과 validation을 나누는 set의 비율을 나타냅니다. 전체 batch갯수중 90%를 training으로, 10%를 validation으로 사용합니다.

<img src="../resources/dataset.jpeg" width="500" alt="split dataset">

x matrix(행렬)는 (`batch크기 x 글자열 길이`)입니다.

In [12]:
def split_data(words, **params):
    batch_size = params.get('batch_size') or 50
    time_steps = params.get('time_steps') or 50
    split_frac = params.get('split_frac') or 0.8
    
    slice_size = batch_size * time_steps
    n_batches = int(len(words) / slice_size)
    x = words[: n_batches*slice_size]
    y = words[1: n_batches*slice_size + 1]
    
    x = np.stack(np.split(x, batch_size))
    y = np.stack(np.split(y, batch_size))
    
    split_idx = int(n_batches*split_frac)
    train_x, train_y= x[:, :split_idx*time_steps], y[:, :split_idx*time_steps]
    val_x, val_y = x[:, split_idx*time_steps:], y[:, split_idx*time_steps:]
    
    return train_x, train_y, val_x, val_y

In [13]:
train_x, train_y, val_x, val_y = split_data(words, **params)

데이터가 나눠졌는지 확인해봅시다.

In [14]:
train_x.shape

(100, 5535)

In [15]:
val_x.shape

(100, 1395)

In [16]:
train_x[:,:10]

array([[16387,  9801, 15499,   848,  4720,  3984, 11012,  6041,  6762,
        16845],
       [ 6782, 17119, 13681,   848, 10027,  1000,  8628,  5808, 19848,
        10465],
       [13176,  3632, 11012, 18521, 19091,   848, 13063,  9801,  4258,
         4634],
       [ 4258,  6724, 10465, 19368, 10465,  4258,  9587,  3881, 10465,
         4258],
       [  266,  6041, 17112,  8144,   848, 17946,  4258,  2073,  6871,
         7560],
       [19081,  6464, 19531,  7933,  2453,  7560, 10465,  3944,  7560,
         5445],
       [ 1281, 10458, 10465,  6230,  2481, 10465,  5439, 10465, 17119,
         4061],
       [ 2579, 18521, 15527,   848,  5116,  9801,  1000,  2579,  5771,
         5439],
       [10465, 17119,  2425,  2615, 15378, 11938, 11964, 17920,  1367,
        17119],
       [ 7777,  9858, 17672, 10138, 18520,   848,  7560, 19961,  3281,
        19289],
       [  848,  7560,  2138,  6871,  6041, 13234,  7315,  6230,  6871,
        19697],
       [17431, 10465,    14, 19289,  1779, 

학습을 할 때 각각 batch를 순서대로 넣어야 하기때문에 batch하나를 가져오는 함수를 만들어봅시다. 각 batch는 (`batch 크기 X time_steps`)입니다.
예를 들면, 우리의 모델이 100개의 문자열에 대해서 학습을 한다면, `time_steps = 100`이 됩니다. 그 다음 batch는 학습한 그 다음 문자열부터 학습됩니다.

In [17]:
def get_batch(arrs, num_steps):
    batch_size, slice_size = arrs[0].shape
    n_batches = int(slice_size/num_steps)
    for b in range(n_batches):
        yield [x[:, b*num_steps: (b+1)*num_steps] for x in arrs]

이제 tensorflow를 이용하여 RNN을 만들어봅시다. tensorflow관련 함수들은 [Tensorflow RNN API](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn)를 참조하시면 됩니다.
##### 참조 링크
- [One-hot vector](https://www.tensorflow.org/api_docs/python/tf/one_hot): 
<img src='../resources/one_hot.png' width="700" alt="one hot encoding">
- [Dropout](https://www.youtube.com/watch?v=NhZVe50QwPM)[참조논문](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf): Dropout은 random하게 특정 node를 0으로 만들어서 back-propagation이 0으로 된 node 이후로 진행되지 않게하여 overfiting을 막아주는 regularization역할을하여 학습을 원할하게합니다. **Advanced Topic: [Batch Normalization](https://arxiv.org/abs/1502.03167)를 추가적으로 공부하시면 overfitting 관련 공부에 도움이 됩니다.**
<img src='../resources/dropout.png' width="700" alt="dropout">
- [Optimizer](https://www.tensorflow.org/versions/r0.12/api_docs/python/train/optimizers)

## TensorBoard에 그래프를 기입

```python
def define_your_model():
    ###
    tf.summary.histogram('histogram', histogram)
    tf.summary.scalar('scalar', scalar)
    ###
    merged = tf.summary.merge_all()
    ###
model = define_your_model()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    ###
    file_writer = tf.summary.FileWriter('logs/', sess.graph)
    file_writer.add_summary(summary_to_record, iteration_index)
    ###
```

In [18]:
def define_rnn_graph(num_classes, **params):
    # parameters
    lstm_size = params.get('lstm_size') or 128
    batch_size = params.get('batch_size') or 50
    time_steps = params.get('time_steps') or 50
    num_layers = params.get('num_layers') or 2
    optimizer_params = params.get('optimizer_params') or {'learning_rate': 1e-3}
    grad_clip = params.get('grad_clip') or 10
    sampling = params.get('sampling') or False
    
    if sampling == True:
        batch_size, time_steps = 1, 1

    tf.reset_default_graph()
    
    # placeholders를 선언합니다.
    # input을 tf.one_hot함수를 이용하여 one_hot vector로 바꿔줍니다.
    with tf.name_scope('inputs'):
        inputs = tf.placeholder(tf.int32, [batch_size, time_steps], name='inputs')
        x_one_hot = tf.one_hot(inputs, num_classes, name='x_one_hot')
    # target도 비슷한 방식으로 진행합니다
    with tf.name_scope('targets'):
        targets = tf.placeholder(tf.int32, [batch_size, time_steps], name='targets')
        y_one_hot = tf.one_hot(targets, num_classes, name='y_one_hot')

        # Loss를 계산하기위해 one_hot vector들의 matrix를 tf.reshape함수를 이용하여 하나의 긴 vector로 바꾸어줍니다.
        y_reshaped = tf.reshape(y_one_hot, [-1, num_classes])
    
    # Dropout을 위한 확률값을 저장하는 place holder
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    # RNN의 한 종류인 LSTM 구현
    with tf.name_scope("RNN_layers"):
        lstm_layers = []
        for _ in range(num_layers):
            lstm = rnn.BasicLSTMCell(lstm_size)
            # rnn.DropoutWrapper를 이용하여 RNN model에 Dropout 추가
            drop = rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
            # LSTM hidden layer 추가, weight sharing
            lstm_layers.append(drop)
        cell = rnn.MultiRNNCell(lstm_layers)

    # tf.nn.dynamic_rnn함수를 이용해 RNN을 실행
    with tf.name_scope("RNN_init_state"):
        initial_state = cell.zero_state(batch_size, tf.float32)
    
    # forward propagation
    with tf.name_scope("RNN_forward"):
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state)
    
    final_state = state

    # Output을 Concatenate한 후에 Reshape합니다.
    with tf.name_scope('reshaper'):
        seq_output = tf.concat(outputs, axis=1,name='seq_output')
        output = tf.reshape(seq_output, [-1, lstm_size], name='graph_output')
    
    # Cost를 계산하기위해 RNN putput을  input으로하는 softmax layer를 제작합니다.
    with tf.name_scope('logits'):
        softmax_w = tf.Variable(tf.truncated_normal((lstm_size, num_classes), stddev=0.1),
                               name='softmax_w')
        softmax_b = tf.Variable(tf.zeros(num_classes), name='softmax_b')
        logits = tf.matmul(output, softmax_w) + softmax_b
        # weights & bias를 histogram으로 작성
        tf.summary.histogram('softmax_w', softmax_w)
        tf.summary.histogram('softmax_b', softmax_b)
        
    with tf.name_scope('predictions'):
        preds = tf.nn.softmax(logits, name='predictions')
        # prediction의 확률값을 histogram으로 작성
        tf.summary.histogram('predictions', preds)
        
    with tf.name_scope('cost'):
        loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped, name='loss')
        cost = tf.reduce_mean(loss, name='cost')
        # cost값을 scalar value로 작성
        tf.summary.scalar('cost', cost)
    
    # 학습을 위한 Optimizer를 정의합니다.
    # 대표적인 optimizer로는 SGD(stocastic gradient descent), Adam, RMSprop 등이 있습니다.
    # Gradient clipping을 통해 gradient값이 매우 큰 경우는 grad_clip값으로 제한합니다.
    with tf.name_scope('train'):
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
        train_op = tf.train.AdamOptimizer(**optimizer_params)
        optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    # summary를 merge합니다.
    merged = tf.summary.merge_all()
    
    # 앞에 선언한 노드들을 모두 Graph로 만들어서 결과로 반환합니다.
    export_nodes = ['inputs', 'targets', 'initial_state', 'final_state',
                    'keep_prob', 'cost', 'preds', 'optimizer','merged']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

## 학습 (Training)

Checkpoint를 저장할 directory를 만듭니다.

In [19]:
if not os.path.isdir('checkpoints/sherlock_word'):
    os.makedirs('checkpoints/sherlock_word')

In [20]:
def load_latest_checkpoint(checkpoint_dir):
    filelist=set([re.search(r'(^i[0-9]+_l[0-9]+_[0-9]+\.[0-9]+)\b',l).group(0) \
                  for l in os.listdir(checkpoint_dir) if re.search(r'(^i[0-9]+_l[0-9]+_[0-9]+\.[0-9]+)\b',l)])
    filelist=list(filelist)
    checkpoint_sorted_by_iterations = sorted(filelist, key=lambda pattern:\
                                    int(re.search(r'^i([0-9]+)_l[0-9]+_[0-9]+\.[0-9]+\b',pattern).group(1)))
    if checkpoint_sorted_by_iterations == []:
        return None
    else:
        return os.path.join(checkpoint_dir,checkpoint_sorted_by_iterations[-1])

In [ ]:
epochs = 20
checkpoint_interval = 50
checkpoint = None
# 기존 checkpoint를 실행하고싶다면 None 대신 checkpoint_path를 넣으면됩니다.
checkpoint = load_latest_checkpoint('checkpoints/sherlock_word')

In [ ]:
train_x, train_y, val_x, val_y = split_data(words, **params)
model = define_rnn_graph(len(dictionary), **params)
saver = tf.train.Saver(max_to_keep=200)
epoch_start = 0
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    # tensorboard 작성을 위한 Filewriter를 만듭니다.
    train_writer = tf.summary.FileWriter('./logs/wordRNN/train', sess.graph)
    test_writer = tf.summary.FileWriter('./logs/wordRNN/test')
       
    n_batches = int(train_x.shape[1]/params['time_steps'])
    iterations = n_batches * epochs
     # 기존의 checkpoint를 읽어서 다시 학습
    if checkpoint:
        try:
            saver.restore(sess, checkpoint)
            iteration=int(re.search(r'\bi([\d]+)_[\w.]+\b',checkpoint).group(1))
            epoch_start = int(iteration/n_batches)
        except:
            print('Cannot read the checkpoint. Set it None.')
            epoch_start = 0
            checkpoint = None
            
    for e in range(epoch_start, epochs):
        # network 학습
        new_state = sess.run(model.initial_state)
        loss = 0
        for i, (x, y) in enumerate(get_batch([train_x, train_y], params['time_steps']), 1):
            iteration = e*n_batches + i
            # training 시간을 기록
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: 0.5,
                    model.initial_state: new_state }
            summary, batch_loss, new_state, _ = sess.run([model.merged, model.cost, \
                                                          model.final_state, model.optimizer], feed_dict=feed)
            loss += batch_loss
            end = time.time()
            print('Epoch {}/{} '.format(e+1, epochs),
                  'Iteration {}/{}'.format(iteration, iterations),
                  'Training loss: {:.4f}'.format(loss/i),
                  '{:.4f} sec/batch'.format((end-start)))
            # summary추가
            train_writer.add_summary(summary, iteration)
            
            if (iteration%checkpoint_interval == 0) or (iteration == iterations):
                # validation loss 확인. dropout의 값을 1로 설정하여 모든 node가 동작하도록 한다.
                val_loss = []
                new_state = sess.run(model.initial_state)
                for x, y in get_batch([val_x, val_y], params['time_steps']):
                    feed = {model.inputs: x,
                            model.targets: y,
                            model.keep_prob: 1.,
                            model.initial_state: new_state}
                    summary, batch_loss, new_state = sess.run([model.merged, model.cost, \
                                                               model.final_state], feed_dict=feed)
                    val_loss.append(batch_loss)
                # summary추가
                test_writer.add_summary(summary, iteration)
                
                print('Validation loss:', np.mean(val_loss),
                      'Saving checkpoint!')
                saver.save(sess, "checkpoints/sherlock_word/i{}_l{}_{:.3f}".format(iteration, params['lstm_size'], np.mean(val_loss)))

Epoch 1/20  Iteration 1/7380 Training loss: 9.9219 0.8294 sec/batch
Epoch 1/20  Iteration 2/7380 Training loss: 9.9206 0.6049 sec/batch
Epoch 1/20  Iteration 3/7380 Training loss: 9.9190 0.5852 sec/batch
Epoch 1/20  Iteration 4/7380 Training loss: 9.9168 0.5695 sec/batch
Epoch 1/20  Iteration 5/7380 Training loss: 9.9136 0.6527 sec/batch
Epoch 1/20  Iteration 6/7380 Training loss: 9.9092 0.6532 sec/batch
Epoch 1/20  Iteration 7/7380 Training loss: 9.9023 0.6217 sec/batch
Epoch 1/20  Iteration 8/7380 Training loss: 9.8906 0.6690 sec/batch
Epoch 1/20  Iteration 9/7380 Training loss: 9.8699 0.7458 sec/batch
Epoch 1/20  Iteration 10/7380 Training loss: 9.8386 0.8175 sec/batch
Epoch 1/20  Iteration 11/7380 Training loss: 9.7890 0.8779 sec/batch
Epoch 1/20  Iteration 12/7380 Training loss: 9.7315 0.9210 sec/batch
Epoch 1/20  Iteration 13/7380 Training loss: 9.6735 0.9240 sec/batch
Epoch 1/20  Iteration 14/7380 Training loss: 9.6187 0.9306 sec/batch
Epoch 1/20  Iteration 15/7380 Training loss

In [ ]:
tf.train.get_checkpoint_state('checkpoints/sherlock_word')

## Sampling

이제 학습된 모델을 이용하여 문장을 만들어봅시다. 학습된 모델이 문장을 만드는 방법은 이전 글자가 주어졌을때, 다음 글자를 예측을 반복적으로 하면서 이루어집니다. 학습된 모델은 주어진 이전 글자에 대해 다음 글자를 확률 값으로 예측을 하게됩니다. 각각의 확률을 적용하여 Random sampling을 하여 새로운 글자가 추가가 되고, 새로운 글자와 이전 state를 이용하여 다음 글자를 예측합니다. 이 과정을 반복하게되면 문장을 만들 수 있습니다.
확률값이 가장 높은 `N`가지중에 하나를 선택하도록 코드를 작성해봅시다.

In [ ]:
def pick_top_n(preds, dictionary_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(dictionary_size, 1, p=p)[0]
    return c

In [ ]:
def sample(checkpoint, n_samples, lstm_size, dictionary_size, prime=["The"]):
    samples = list(prime)
    model = define_rnn_graph(dictionary_size, **{'lstm_size':lstm_size, 'sampling':True})
    saver = tf.train.Saver()
    with tf.Session(config=config) as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = word_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, dictionary_size)
        samples.append(int_to_word[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, dictionary_size)
            # dealing with special char not word.
            if c in list('.,\?!\'"'):
                if samples[-1] in list('.,\?!\'"'):
                    samples.append(int_to_word[c])
                else:
                    # concatnate the c letter to the previous word
                    samples[-1] = samples[-1] + c                    
            elif c in list('0123456789'):
                if samples[-1] in list('0123456789'):
                    # concatnate the c letter to the previous word
                    samples[-1] = samples[-1] + c
                else:
                    samples.append(int_to_word[c])
            else:
                samples.append(int_to_word[c])
        
    return ' '.join(samples)

Validation Loss가 가장 작은 모델을 포함한 여러 모델을 이용하여 문장을 만들어봅시다.

In [ ]:
all_checkpoints=re.findall(r'\b([\w/]+_([\d.]+))\b',str(tf.train.get_checkpoint_state('checkpoints/sherlock')),re.IGNORECASE)
all_checkpoints_sorted_by_valloss = sorted(all_checkpoints, key=lambda tup: float(tup[1]))

In [ ]:
all_checkpoints_sorted_by_valloss[:10]

In [ ]:
n_samples = 100

10번째 checkpoint의 prediction 결과

In [ ]:
checkpoint = all_checkpoints[10][0]
samp = sample(checkpoint, n_samples, params['lstm_size'], len(dictionary), prime=[int_to_word[0]])
print('<<{}>>\n'.format(checkpoint)+samp)
print('='*100)

validation loss가 가장 작은 checkpoint의 prediction 결과

In [ ]:
for checkpoint, _ in all_checkpoints_sorted_by_valloss[:2]:
    samp = sample(checkpoint, n_samples, params['lstm_size'], len(charset), prime=[int_to_word[0]])
    print('<<{}>>\n'.format(checkpoint)+samp)
    print('='*100)

In [ ]:
for checkpoint, _ in all_checkpoints_sorted_by_valloss[:1]:
    samp = sample(checkpoint, n_samples, params['lstm_size'], len(charset), prime=[int_to_word[0]])
    print('<<{}>>\n'.format(checkpoint)+samp)
    print('='*100)

마지막 checkpoint의 prediction 결과

In [ ]:
checkpoint = tf.train.latest_checkpoint('checkpoints/sherlock')
samp = sample(checkpoint, 1000, params['lstm_size'], len(charset), prime=[int_to_word[0]])
print(samp)

## Tensorboard
log directory를 설정해주고 실행합니다.
```bash
$ tensorboard --logdir='logs/'
```